# 🌊 Skoðun hafgagna frá Copernicus í Google Colab

**Hér er að neðan eru gagnvirkar leiðbeiningar sem Náttúrufræðistofnun hefur sett upp til að auðvelda aðgang að gagnasettum frá Hafþjónustu Copernicus (Copernicus Marine).**

---

### Yfirlit
Leiðbeiningarnar tengjast beint við hafþjónustu Copernicus og safna saman upplýsingum um öll gagnasett sem þar eru í boði. Með Python-forritun verður sjálfkrafa til skjal með yfirliti yfir gagnasettin og lýsingu á þeim.

Forritið skráir sig inn í Copernicus Marine með notandaupplýsingum og nýtir API-þjónustu til að sækja lýsigögn.

Heildarlisti yfir tiltæk gögn: https://data.marine.copernicus.eu/products

### Notkun
1. Opnið leiðbeiningarnar í Google Colab eða annarri skýjaþjónustu.
2. Útbúið aðgang að Copernicus Marine: https://data.marine.copernicus.eu/register
3. Setjið inn réttar notandaupplýsingar við innskráningu (Login) til að fá aðgang að gögnunum.
4. Keyrið kóðareitina í réttri röð (án þess að sleppa neinum reitum) til að skoða lýsingar á gagnasettum og tiltækri þjónustu, hlaða þeim niður, vinna úr þeim og birta. 

| Hægt er að prófa kóðann í ókeypis skýjaumhverfi: | [![Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/lmi/Copernicus/blob/master/Marine/Overview/ConnectMarine_IS.ipynb)

---

### Höfundaréttur og tilvísanir
- **Notkunarleyfi**: Leiðbeiningarnar er gefnar út samkvæmt alþjóðlegu höfundarréttarleyfi **CC BY 4.0**.
- **Tilvísun**: "*Heimild: Náttúrufræðistofnun*"
- **Höfundur**: Marco Pizzolato
- **Uppruni gagna**: [Copernicus Marine Service](https://marine.copernicus.eu/)
- **Forritunarmál**: Python

---

**Með þessum leiðbeiningum er hægt að skoða og skrá gagnasett frá hafþjónustu Copernicus á sjálfvirkan hátt, sem auðveldar aðgang að hafgögnum.**

---
Skjalið er unnið fyrir Blue North verkefni:
<p align="center">
  <a href="https://copernicus.gis.is/homepage/index.php/home/hafthjonusta-copernicus/blue-north-verkefnid/">
    <img src="https://atlas.lmi.is/gn/myndir/BlueNorth_Logo.png"
         width="120"
         alt="Blue North Logo">
  </a>
</p>

Þessi verkefni er studd af :
<p align="center">
  <a href="">
    <img src="https://atlas.lmi.is/gn/myndir/logo_cmems_bn.png"
         width="1000"
         alt="CMEMS Logo">
  </a>
</p>

## Uppsetning og flytja inn gögn

In [ ]:
!pip install -q copernicusmarine pandas tabulate Pillow mdutils leafmap rioxarray xarray rasterio pyproj localtileserver

In [ ]:
import os
import requests
from PIL import Image
from mdutils.mdutils import MdUtils
from mdutils.tools import Html

## Innskráning í hafþjónustu Copernicus

Útbúið aðgang að þjónustunni hér: https://data.marine.copernicus.eu/register

In [ ]:
import copernicusmarine as cm
import getpass

username = input("Enter your Copernicus Marine username: ")
password = getpass.getpass("Enter your Copernicus Marine password: ")

cm.login(username=username, password=password)

## Sækja gagnaskrá

In [ ]:
catalog = cm.describe()

In [ ]:
# Inspect the structure of the first product and its datasets
first_product = catalog.products[0]
print(first_product.model_dump())
print(first_product.datasets)

## Vista gagnaskrá í Markdown-sniði

In [ ]:
# Initialize the Markdown file
md_file = MdUtils(file_name='Copernicus_Marine_Products_and_Datasets')

# Iterate through products
for product in catalog.products:
    product_title = product.title
    product_id = product.product_id
    description = product.description
    thumbnail_url = product.thumbnail_url

        # Add thumbnail image if available
    if thumbnail_url:
        # Resize the image to 100x100 pixels using HTML
        img_tag = Html.image(path=thumbnail_url, size='300x200')
        md_file.new_paragraph(img_tag)

    # Add an empty line after the image
    md_file.new_paragraph('\n')

    # Add a header for the product
    md_file.new_header(level=1, title=product_title)

    # Add product ID
    md_file.new_paragraph(f'**Product ID:** {product_id}')

    # Add description
    md_file.new_paragraph(f'**Description:** {description}')

    # Add an empty line after the image
    md_file.new_paragraph('\n')

In [ ]:
# Create the Markdown file
md_file.create_md_file()

## Birta yfirlitið sem búið var til

In [ ]:
from IPython.display import display, Markdown

In [ ]:
# Display the generated markdown file at the end of the notebook
n_lines = 100  # how many lines to display
with open("Copernicus_Marine_Products_and_Datasets.md", "r") as file:
    content = "".join([next(file) for _ in range(n_lines)])
display(Markdown(content + "\n\n... *truncated*"))

## Hlaða niður gögnum

In [ ]:
import datetime as dt
# Product example: ARCTIC_ANALYSISFORECAST_BGC_002_004
DATASET_ID = "cmems_mod_arc_bgc_anfc_ecosmo_P1D-m"  #  ⟵ from product page (Select one of you choice)

# Small AOI covering Iceland and nearby Arctic (adjust as you wish)
bbox = dict(
    minimum_longitude=-40.0, maximum_longitude=40.0,
    minimum_latitude=60.0,  maximum_latitude=85.0
)

# Use "latest" day that is surely produced (yesterday is safe for NRT)
end = dt.datetime.utcnow().date()
start = end - dt.timedelta(days=1)
time_sel = dict(
    start_datetime=start.isoformat(),
    end_datetime=end.isoformat()
)

# Name of the file we download
out_zarr = "/content/arctic_bgc_chl.zarr"

In [ ]:
# We'll ask for "chl*" but stay robust: if variable name differs, we'll detect it later from the file itself.
# Request only surface layer (0 m). Many CMEMS BGC datasets use "depth" in meters.
# Download as Zarr (directory)
cm.subset(
    dataset_id=DATASET_ID,
    variables=None,                      # we'll auto-detect the chl var after download
    **bbox,
    minimum_depth=0, maximum_depth=0,    # surface layer
    **time_sel,
    output_filename=out_zarr             # <-- writes a Zarr store
)

## Breyta gögnum í TIFF-snið (til að sýna á korti)

In [ ]:
# Open the Zarr store with xarray
import xarray as xr
import rioxarray as rxr

# If your store is consolidated, set consolidated=True. If unsure, xarray will figure it out.
ds = xr.open_zarr(out_zarr)

# Find a chlorophyll-like variable
candidates = [v for v in ds.data_vars if ("chl" in v.lower()) or ("chlor" in v.lower())]
if not candidates:
    raise ValueError(f"No chlorophyll-like variable found in {list(ds.data_vars)}")
var = candidates[0]
da = ds[var]

# Select the last time slice (if time exists)
if "time" in da.dims and da.sizes.get("time", 0) > 0:
    da = da.isel(time=-1)

# Write a GeoTIFF, inferring CRS from CF metadata where possible
def write_geotiff_from_da(da, tif_path="arctic_bgc_chl.tif"):
    # Case A: we have geographic lon/lat coords
    has_lat = any(n in da.coords for n in ("lat", "latitude"))
    has_lon = any(n in da.coords for n in ("lon", "longitude"))
    if has_lat and has_lon:
        lat_name = "lat" if "lat" in da.coords else "latitude"
        lon_name = "lon" if "lon" in da.coords else "longitude"
        da = da.rename({lat_name: "lat", lon_name: "lon"})

        # rioxarray expects x,y order and 1D coords
        da = da.rio.set_spatial_dims(x_dim="lon", y_dim="lat", inplace=False)
        da = da.rio.write_crs(4326, inplace=False)  # EPSG:4326
        da.rio.to_raster(tif_path)
        return tif_path

    # Case B: projected grid with x/y-ish dims; try to get CRS from CF grid_mapping
    spatial_dims = [d for d in da.dims if d.lower() in ("x","y","rlon","rlat","cols","rows")]
    if len(spatial_dims) >= 2:
        # choose an order that prefers x-like then y-like
        order = []
        for cand in ("x","rlon","cols"):
            if cand in da.dims: order.append(cand)
        for cand in ("y","rlat","rows"):
            if cand in da.dims: order.append(cand)
        if len(order) != 2:
            order = da.dims[:2]  # fallback

        da = da.rename({order[0]: "x", order[1]: "y"})
        da = da.rio.set_spatial_dims(x_dim="x", y_dim="y", inplace=False)

        # Try to attach CRS from CF grid mapping variable
        gm = getattr(da.rio, "grid_mapping", None)
        crs_written = False
        if gm:
            # try 'spatial_ref' attr (WKT) first; else try CRS from rioxarray if present
            wkt = da[gm].attrs.get("spatial_ref")
            if wkt:
                da = da.rio.write_crs(wkt, inplace=False)
                crs_written = True
            else:
                try:
                    # some datasets expose a parsed CRS via .rio.crs on the mapping var
                    _ = da[gm].rio.crs  # may raise
                    da = da.rio.write_crs(da[gm].rio.crs, inplace=False)
                    crs_written = True
                except Exception:
                    pass

            # (Optional) very light heuristic if only grid_mapping_name is present
            if not crs_written:
                gm_name = da[gm].attrs.get("grid_mapping_name", "").lower()
                # DO NOT default to EPSG:3857 (Web Mercator). It's wrong for polar products.
                # Common arctic choices:
                # - polar_stereographic: EPSG:3413 (NSIDC NSIDC Sea Ice), varies by standard_parallel
                # - lambert_azimuthal_equal_area: EPSG:3995
                if "lambert_azimuthal_equal_area" in gm_name:
                    da = da.rio.write_crs("EPSG:3995", inplace=False)
                    crs_written = True
                # If it's polar_stereographic we can't safely guess the exact EPSG without attrs;
                # better to leave unset than to write a wrong CRS.

        # As a last resort: don't invent a CRS—writing a wrong one is worse than none.
        da.rio.to_raster(tif_path)
        return tif_path

    raise RuntimeError("Could not determine geospatial coordinates/CRS to write GeoTIFF.")

tif_path = write_geotiff_from_da(da, "arctic_bgc_chl.tif")
tif_path

## Birta gögnin á korti

In [ ]:
import leafmap

# Initialize map
m = leafmap.Map(center=[70, -20], zoom=3)

# Add background layer
m.add_basemap("Esri.OceanBasemap")

# Add our downloaded raster
m.add_raster(
    tif_path,
    layer_name="Chlorophyll-a (surface)",
    zoom_to_layer=True,
    colormap="viridis",
    vmin=0, # Min value for color scale
    vmax=10 # Max value for color scale
)

# Display map
m

## Sýna gögn beint af vefnum án þess að hlaða þeim niður

In [ ]:
import leafmap

# The layer identifier you can find them all here: https://data.marine.copernicus.eu/products
LAYER_ID = ("ARCTIC_ANALYSISFORECAST_BGC_002_004/"
            "cmems_mod_arc_bgc_anfc_ecosmo_P1D-m_202105/chl")

# Pick any of the listed times;
ISO_TIME = "2025-07-01T00:00:00Z"

# Optional: elevation (depth, in meters). The default in the XML is ~ -0.494 m (surface layer in z*)
ELEVATION = "-0.4940253794193268"   # or omit the &ELEVATION param to use the default

# Choose a TileMatrixSet that matches your basemap (Web Mercator or WGS84)
TMS = "EPSG:3857"

# Server-side styling. If viridis isn’t available on this layer, switch to 'cmap:matter' or use 'logScale' variant.
STYLE = "cmap:viridis"      # fallback: "cmap:matter" or "cmap:matter,logScale"

# Make URL
WMTS_URL = (
    "https://wmts.marine.copernicus.eu/teroWmts"
    "?SERVICE=WMTS&REQUEST=GetTile&VERSION=1.0.0"
    f"&LAYER={LAYER_ID}"
    "&FORMAT=image/png"
    f"&TILEMATRIXSET={TMS}"
    "&TILEMATRIX={z}&TILEROW={y}&TILECOL={x}"
    f"&time={ISO_TIME}"
    f"&STYLE={STYLE}"
    f"&ELEVATION={ELEVATION}"
)

# Prepare map
m = leafmap.Map(center=[30, 0], zoom=2)

# Add our WMTS layer
m.add_tile_layer(
    url=WMTS_URL,
    name="CMEMS Zooplankton (monthly)",
    attribution="Copernicus Marine"
)
m

Munurinn er sá að ekki er hægt að vinna með gögn sem skoðuð eru beint á netinu í gegnum WMTS-þjónustu, en þegar þau eru sótt er hægt að tengja þau við önnur gögn eða skoða einstök gildi í þeim.